In [ ]:
!pip install -qU pinecone-client openai langchain gradio

In [ ]:
import gradio as gr
import pinecone
import openai
import warnings

from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

openai.api_key = ""
PINECONE_API_KEY = ""  # find next to api key in console
PINECONE_ENV = ""  # find next to api key in console

EMBEDDING_MODEL_NAME = 'text-embedding-ada-002'
warnings.filterwarnings('ignore')

embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL_NAME,
    openai_api_key=openai.api_key
)

index_name = 'semantic-search-openai'
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )
# connect to index
index = pinecone.Index(index_name)
print('Pinecone index status is', index.describe_index_stats())

text_field = "text"
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

MODEL_NAME = 'gpt-3.5-turbo'
# completion llm
llm = ChatOpenAI(
    openai_api_key=openai.api_key,
    model_name=MODEL_NAME,
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

Now test the SEC file reader

In [ ]:
print(qa("What is the revanues for Tesla for 2022?"))

Build a Web UI using Gradio!!

In [ ]:
import gradio as gr

def query_vector_db(query):
    return qa(query)

sec_file_query_demo_app = gr.Interface(
    fn=query_vector_db,
    inputs=gr.Textbox(lines=1, placeholder="Please Enter the SEC file search query here !"),
    outputs="text",
)
sec_file_query_demo_app.launch()